# API 기능 테스트 노트북

이 노트북은 Review Analysis API의 모든 기능을 테스트하기 위한 것입니다.

## 테스트 항목
1. 환경 설정 및 초기화
2. 샘플 데이터 생성
3. 벡터 데이터 업로드
4. 감성 분석 테스트
5. 벡터 검색 테스트
6. 리뷰 요약 테스트
7. 강점 추출 테스트
8. 리뷰 Upsert 테스트 (개별 및 배치)
9. 리뷰 삭제 테스트 (개별 및 배치)
10. 이미지 리뷰 검색 테스트


## 1. 환경 설정 및 초기화


In [1]:
# 필요한 라이브러리 import
import sys
import os
import json
import requests
from typing import Dict, List, Any

# 환경 변수 설정 (필요시)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"
# os.environ["QDRANT_URL"] = "http://localhost:6333"  # 또는 ":memory:"

# API 기본 URL 설정
BASE_URL = "http://localhost:8000"

# 헬스 체크
def check_health():
    """API 서버 상태 확인"""
    try:
        response = requests.get(f"{BASE_URL}/health")
        if response.status_code == 200:
            print("✅ API 서버가 정상적으로 실행 중입니다.")
            print(f"   상태: {response.json()}")
            return True
        else:
            print(f"❌ API 서버 응답 오류: {response.status_code}")
            return False
    except Exception as e:
        print(f"❌ API 서버에 연결할 수 없습니다: {str(e)}")
        print(f"   서버가 실행 중인지 확인하세요: {BASE_URL}")
        return False

# 헬스 체크 실행
if not check_health():
    print("\n⚠️ API 서버를 먼저 실행하세요:")
    print("   uvicorn src.api.main:app --host 0.0.0.0 --port 8000 --reload")
    print("   또는")
    print("   python app.py")


✅ API 서버가 정상적으로 실행 중입니다.
   상태: {'status': 'healthy', 'version': '1.0.0'}


## 2. 샘플 데이터 생성


In [2]:
# 테스트용 샘플 데이터 생성
sample_data = {
    "restaurants": [
        {
            "restaurant_id": "res_1234",
            "restaurant_name": "비즐",
            "reviews": [
                {
                    "review_id": "rev_3001",
                    "user_id": "user_2001",
                    "datetime": "2026-01-03 12:10:00",
                    "group": "카카오",
                    "review": "점심시간이라 사람이 많았지만 생각보다 빨리 나왔다. 맛도 좋고 서비스도 친절해요!",
                    "images": {"url": "http://localhost:8000/bizzle_image1.jpeg"},
                    "version": 1
                },
                {
                    "review_id": "rev_3002",
                    "user_id": "user_2002",
                    "datetime": "2026-01-03 12:12:00",
                    "group": "네이버",
                    "review": "가츠동은 괜찮았는데 다른 메뉴는 좀 애매했다. 다음엔 다른 메뉴를 시도해볼게요.",
                    "images": {"url": "http://localhost:8000/bizzle_image2.jpeg"},
                    "version": 1
                },
                {
                    "review_id": "rev_3003",
                    "user_id": "user_2003",
                    "datetime": "2026-01-04 18:45:00",
                    "group": "카카오",
                    "review": "직원 응대가 그날그날 다른 느낌이다. 일관성이 부족해요.",
                    "images": [],
                    "version": 1
                },
                {
                    "review_id": "rev_3004",
                    "user_id": "user_2004",
                    "datetime": "2026-01-05 13:00:00",
                    "group": "네이버",
                    "review": "음식은 맛있을 때도 있지만 오늘은 좀 짰다. 조금 아쉬웠어요.",
                    "images": [],
                    "version": 1
                },
                {
                    "review_id": "rev_3005",
                    "user_id": "user_2005",
                    "datetime": "2026-01-06 19:20:00",
                    "group": "카카오",
                    "review": "웨이팅이 길 줄 알았는데 회전이 빨라서 괜찮았다. 맛도 좋고 가성비도 좋아요!",
                    "images": [],
                    "version": 1
                },
                {
                    "review_id": "rev_3006",
                    "user_id": "user_2006",
                    "datetime": "2026-01-06 19:25:00",
                    "group": "네이버",
                    "review": "서비스는 기대 안 하는 게 정신 건강에 좋다. 하지만 음식은 나쁘지 않아요.",
                    "images": [],
                    "version": 1
                }
            ]
        },
        {
            "restaurant_id": "res_1235",
            "restaurant_name": "시올돈",
            "reviews": [
                {
                    "review_id": "rev_4001",
                    "user_id": "user_2101",
                    "datetime": "2026-02-03 18:00:00",
                    "group": "카카오",
                    "review": "음식 맛은 무난하고 실패는 없는 편이다. 괜찮은 식당이에요.",
                    "images": {"url": "http://localhost:8000/sioldon_image1.jpeg"},
                    "version": 1
                },
                {
                    "review_id": "rev_4002",
                    "user_id": "user_2102",
                    "datetime": "2026-02-03 18:20:00",
                    "group": "네이버",
                    "review": "웨이팅이 너무 길어서 중간에 포기할 뻔했다. 하지만 맛은 좋았어요.",
                    "images": {"url": "http://localhost:8000/sioldon_image2.jpeg"},
                    "version": 1
                },
                {
                    "review_id": "rev_4003",
                    "user_id": "user_2103",
                    "datetime": "2026-02-04 19:00:00",
                    "group": "카카오",
                    "review": "가격 대비 만족스러운 맛이에요. 자주 방문할 것 같아요!",
                    "images": [],
                    "version": 1
                }
            ]
        }
    ]
}

print(f"✅ 샘플 데이터 생성 완료")
print(f"   레스토랑 수: {len(sample_data['restaurants'])}")
for restaurant in sample_data['restaurants']:
    print(f"   - {restaurant['restaurant_name']} ({restaurant['restaurant_id']}): {len(restaurant['reviews'])}개 리뷰")


✅ 샘플 데이터 생성 완료
   레스토랑 수: 2
   - 비즐 (res_1234): 6개 리뷰
   - 시올돈 (res_1235): 3개 리뷰


## 3. 벡터 데이터 업로드


In [3]:
# 벡터 데이터 업로드
def upload_vector_data(data: Dict[str, Any]) -> bool:
    """벡터 데이터 업로드"""
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/vector/upload",
            json={"data": data}
        )
        
        if response.status_code == 200:
            result = response.json()
            print(f"✅ 벡터 데이터 업로드 성공")
            print(f"   포인트 수: {result['points_count']}")
            print(f"   컬렉션: {result['collection_name']}")
            return True
        else:
            print(f"❌ 업로드 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return False
    except Exception as e:
        print(f"❌ 업로드 중 오류: {str(e)}")
        return False

# 데이터 업로드 실행
upload_success = upload_vector_data(sample_data)


✅ 벡터 데이터 업로드 성공
   포인트 수: 9
   컬렉션: reviews_collection


## 4. 감성 분석 테스트


In [4]:
# 감성 분석 테스트
def test_sentiment_analysis():
    """감성 분석 테스트"""
    reviews = [
        "점심시간이라 사람이 많았지만 생각보다 빨리 나왔다. 맛도 좋고 서비스도 친절해요!",
        "가츠동은 괜찮았는데 다른 메뉴는 좀 애매했다.",
        "직원 응대가 그날그날 다른 느낌이다.",
        "음식은 맛있을 때도 있지만 오늘은 좀 짰다.",
        "웨이팅이 길 줄 알았는데 회전이 빨라서 괜찮았다. 맛도 좋고 가성비도 좋아요!",
        "서비스는 기대 안 하는 게 정신 건강에 좋다."
    ]
    
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/sentiment/analyze",
            json={
                "reviews": reviews,
                "restaurant_name": "비즐",
                "restaurant_id": "res_1234",
                "score_threshold": 0.8
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ 감성 분석 성공")
            print(f"   긍정 비율: {result['positive_ratio']}%")
            print(f"   부정 비율: {result['negative_ratio']}%")
            print(f"   긍정 개수: {result['positive_count']}")
            print(f"   부정 개수: {result['negative_count']}")
            print(f"   총 개수: {result['total_count']}")
            print(f"   LLM 재분류: {result['llm_reclassified_count']}개")
            return result
        else:
            print(f"❌ 감성 분석 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return None
    except Exception as e:
        print(f"❌ 감성 분석 중 오류: {str(e)}")
        return None

# 감성 분석 실행
sentiment_result = test_sentiment_analysis()


✅ 감성 분석 성공
   긍정 비율: 50%
   부정 비율: 50%
   긍정 개수: 3
   부정 개수: 3
   총 개수: 6
   LLM 재분류: 4개


## 5. 벡터 검색 테스트


In [5]:
# 벡터 검색 테스트
def test_vector_search():
    """벡터 검색 테스트"""
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/vector/search/similar",
            json={
                "query_text": "맛있다 좋다 만족",
                "restaurant_id": "res_1234",
                "limit": 5,
                "min_score": 0.0
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ 벡터 검색 성공")
            print(f"   검색 결과: {result['total']}개")
            
            for i, item in enumerate(result['results'][:3], 1):
                print(f"\n   결과 {i}:")
                print(f"     리뷰: {item['payload']['review'][:50]}...")
                print(f"     점수: {item['score']:.3f}")
                print(f"     레스토랑: {item['payload']['restaurant_name']}")
            
            return result
        else:
            print(f"❌ 벡터 검색 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return None
    except Exception as e:
        print(f"❌ 벡터 검색 중 오류: {str(e)}")
        return None

# 벡터 검색 실행
search_result = test_vector_search()


✅ 벡터 검색 성공
   검색 결과: 5개

   결과 1:
     리뷰: 웨이팅이 길 줄 알았는데 회전이 빨라서 괜찮았다. 맛도 좋고 가성비도 좋아요!...
     점수: 0.654
     레스토랑: 비즐

   결과 2:
     리뷰: 점심시간이라 사람이 많았지만 생각보다 빨리 나왔다. 맛도 좋고 서비스도 친절해요!...
     점수: 0.590
     레스토랑: 비즐

   결과 3:
     리뷰: 음식은 맛있을 때도 있지만 오늘은 좀 짰다. 조금 아쉬웠어요....
     점수: 0.572
     레스토랑: 비즐


## 6. 리뷰 요약 테스트


In [6]:
# 리뷰 요약 테스트
def test_summarize():
    """리뷰 요약 테스트"""
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/llm/summarize",
            json={
                "restaurant_id": "res_1234",
                "positive_query": "맛있다 좋다 만족",
                "negative_query": "맛없다 별로 불만",
                "limit": 10,
                "min_score": 0.0
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ 리뷰 요약 성공")
            print(f"\n   긍정 요약:")
            print(f"   {result['positive_summary']}")
            print(f"\n   부정 요약:")
            print(f"   {result['negative_summary']}")
            print(f"\n   전체 요약:")
            print(f"   {result['overall_summary']}")
            print(f"\n   긍정 리뷰: {result['positive_count']}개")
            print(f"   부정 리뷰: {result['negative_count']}개")
            return result
        else:
            print(f"❌ 리뷰 요약 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return None
    except Exception as e:
        print(f"❌ 리뷰 요약 중 오류: {str(e)}")
        return None

# 리뷰 요약 실행
summarize_result = test_summarize()


✅ 리뷰 요약 성공

   긍정 요약:
   웨이팅이 빠르고 가성비가 좋으며, 서비스도 친절하다.

   부정 요약:
   음식의 맛이 일관되지 않고, 일부 메뉴는 애매하며, 직원 응대가 일관성이 없다.

   전체 요약:
   가성비와 빠른 서비스가 장점이지만, 음식 맛과 직원 일관성이 개선 필요.

   긍정 리뷰: 6개
   부정 리뷰: 6개


## 7. 강점 추출 테스트


In [7]:
# 강점 추출 테스트
def test_extract_strengths():
    """강점 추출 테스트"""
    try:
        # 특정 레스토랑과 비교
        response = requests.post(
            f"{BASE_URL}/api/v1/llm/extract/strengths",
            json={
                "target_restaurant_id": "res_1234",
                "comparison_restaurant_ids": ["res_1235"],  # None이면 타겟 제외한 모든 레스토랑과 자동 비교
                "query": "맛있다 좋다 만족",
                "limit": 5,
                "min_score": 0.0
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ 강점 추출 성공")
            print(f"\n   강점 요약:")
            print(f"   {result['strength_summary']}")
            print(f"\n   타겟 리뷰: {result['target_count']}개")
            print(f"   비교 리뷰: {result['comparison_count']}개")
            return result
        else:
            print(f"❌ 강점 추출 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return None
    except Exception as e:
        print(f"❌ 강점 추출 중 오류: {str(e)}")
        return None

# 강점 추출 실행
strengths_result = test_extract_strengths()


✅ 강점 추출 성공

   강점 요약:
   분석 실패

   타겟 리뷰: 5개
   비교 리뷰: 3개


## 8. 리뷰 Upsert 테스트


In [8]:
# 리뷰 Upsert 테스트 (개별)
def test_upsert_review():
    """개별 리뷰 Upsert 테스트"""
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/vector/reviews/upsert",
            json={
                "restaurant_id": "res_1234",
                "restaurant_name": "비즐",
                "review": {
                    "review_id": "rev_3007",
                    "review": "새로운 리뷰입니다! 맛있어요!",
                    "user_id": "user_2007",
                    "datetime": "2026-01-07 12:00:00",
                    "group": "카카오",
                    "version": 1
                },
                "update_version": None  # 중복 방지
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ 리뷰 Upsert 성공")
            print(f"   작업: {result['action']}")
            print(f"   리뷰 ID: {result['review_id']}")
            print(f"   버전: {result['version']}")
            return result
        else:
            print(f"❌ 리뷰 Upsert 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return None
    except Exception as e:
        print(f"❌ 리뷰 Upsert 중 오류: {str(e)}")
        return None

# 개별 Upsert 실행
upsert_result = test_upsert_review()


✅ 리뷰 Upsert 성공
   작업: updated
   리뷰 ID: rev_3007
   버전: 2


In [9]:
# 리뷰 배치 Upsert 테스트
def test_upsert_reviews_batch():
    """배치 리뷰 Upsert 테스트"""
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/vector/reviews/upsert/batch",
            json={
                "restaurant_id": "res_1234",
                "restaurant_name": "비즐",
                "reviews": [
                    {
                        "review_id": f"rev_{i:04d}",
                        "review": f"배치 리뷰 {i}입니다. 맛있어요!",
                        "user_id": f"user_{i}",
                        "datetime": "2026-01-07 12:00:00",
                        "group": "카카오",
                        "version": 1
                    }
                    for i in range(3008, 3013)  # 5개 리뷰
                ],
                "batch_size": 32
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ 배치 리뷰 Upsert 성공")
            print(f"   총 처리: {result['total']}개")
            print(f"   성공: {result['success_count']}개")
            print(f"   실패: {result['error_count']}개")
            
            # 결과 샘플 출력
            for r in result['results'][:3]:
                print(f"   - {r['review_id']}: {r['action']} (version {r['version']})")
            
            return result
        else:
            print(f"❌ 배치 리뷰 Upsert 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return None
    except Exception as e:
        print(f"❌ 배치 리뷰 Upsert 중 오류: {str(e)}")
        return None

# 배치 Upsert 실행
batch_upsert_result = test_upsert_reviews_batch()


✅ 배치 리뷰 Upsert 성공
   총 처리: 5개
   성공: 5개
   실패: 0개
   - rev_3008: updated (version 2)
   - rev_3009: updated (version 2)
   - rev_3010: updated (version 2)


## 9. 리뷰 삭제 테스트


In [10]:
# 리뷰 삭제 테스트 (개별)
def test_delete_review():
    """개별 리뷰 삭제 테스트"""
    try:
        response = requests.delete(
            f"{BASE_URL}/api/v1/vector/reviews/delete",
            json={
                "restaurant_id": "res_1234",
                "review_id": "rev_3007"  # 위에서 추가한 리뷰
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ 리뷰 삭제 성공")
            print(f"   작업: {result['action']}")
            print(f"   리뷰 ID: {result['review_id']}")
            return result
        else:
            print(f"❌ 리뷰 삭제 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return None
    except Exception as e:
        print(f"❌ 리뷰 삭제 중 오류: {str(e)}")
        return None

# 개별 삭제 실행
delete_result = test_delete_review()


✅ 리뷰 삭제 성공
   작업: deleted
   리뷰 ID: rev_3007


In [11]:
# 리뷰 배치 삭제 테스트
def test_delete_reviews_batch():
    """배치 리뷰 삭제 테스트"""
    try:
        response = requests.delete(
            f"{BASE_URL}/api/v1/vector/reviews/delete/batch",
            json={
                "restaurant_id": "res_1234",
                "review_ids": ["rev_3008", "rev_3009", "rev_3010"]  # 위에서 추가한 리뷰들
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ 배치 리뷰 삭제 성공")
            print(f"   총 처리: {result['total']}개")
            print(f"   삭제: {result['deleted_count']}개")
            print(f"   미발견: {result['not_found_count']}개")
            
            # 결과 샘플 출력
            for r in result['results']:
                print(f"   - {r['review_id']}: {r['action']}")
            
            return result
        else:
            print(f"❌ 배치 리뷰 삭제 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return None
    except Exception as e:
        print(f"❌ 배치 리뷰 삭제 중 오류: {str(e)}")
        return None

# 배치 삭제 실행
batch_delete_result = test_delete_reviews_batch()


✅ 배치 리뷰 삭제 성공
   총 처리: 3개
   삭제: 3개
   미발견: 0개
   - rev_3008: deleted
   - rev_3009: deleted
   - rev_3010: deleted


## 10. 이미지 리뷰 검색 테스트


In [12]:
# 이미지 리뷰 검색 테스트
def test_search_reviews_with_images():
    """이미지가 있는 리뷰 검색 테스트"""
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/vector/search/with-images",
            json={
                "query_text": "맛있다",
                "limit": 10,
                "min_score": 0.0
            }
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ 이미지 리뷰 검색 성공")
            print(f"   검색 결과: {result['total']}개")
            
            for i, item in enumerate(result['results'][:3], 1):
                print(f"\n   결과 {i}:")
                print(f"     리뷰: {item['payload']['review'][:50]}...")
                print(f"     점수: {item['score']:.3f}")
                print(f"     이미지: {item['image_urls']}")
                print(f"     레스토랑: {item['payload']['restaurant_name']}")
            
            return result
        else:
            print(f"❌ 이미지 리뷰 검색 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return None
    except Exception as e:
        print(f"❌ 이미지 리뷰 검색 중 오류: {str(e)}")
        return None

# 이미지 리뷰 검색 실행
images_result = test_search_reviews_with_images()


✅ 이미지 리뷰 검색 성공
   검색 결과: 4개

   결과 1:
     리뷰: 점심시간이라 사람이 많았지만 생각보다 빨리 나왔다. 맛도 좋고 서비스도 친절해요!...
     점수: 0.564
     이미지: ['http://localhost:8000/bizzle_image1.jpeg']
     레스토랑: 비즐

   결과 2:
     리뷰: 음식 맛은 무난하고 실패는 없는 편이다. 괜찮은 식당이에요....
     점수: 0.544
     이미지: ['http://localhost:8000/sioldon_image1.jpeg']
     레스토랑: 시올돈

   결과 3:
     리뷰: 가츠동은 괜찮았는데 다른 메뉴는 좀 애매했다. 다음엔 다른 메뉴를 시도해볼게요....
     점수: 0.518
     이미지: ['http://localhost:8000/bizzle_image2.jpeg']
     레스토랑: 비즐


## 11. 레스토랑 리뷰 조회 테스트


In [13]:
# 레스토랑 리뷰 조회 테스트
def test_get_restaurant_reviews():
    """레스토랑 리뷰 조회 테스트"""
    try:
        response = requests.get(
            f"{BASE_URL}/api/v1/vector/restaurants/res_1234/reviews"
        )
        
        if response.status_code == 200:
            result = response.json()
            print("✅ 레스토랑 리뷰 조회 성공")
            print(f"   레스토랑 ID: {result['restaurant_id']}")
            print(f"   총 리뷰 수: {result['total']}개")
            
            # 리뷰 샘플 출력
            for i, review in enumerate(result['reviews'][:3], 1):
                print(f"\n   리뷰 {i}:")
                print(f"     ID: {review.get('review_id')}")
                print(f"     내용: {review.get('review', '')[:50]}...")
                print(f"     사용자: {review.get('user_id')}")
            
            return result
        else:
            print(f"❌ 레스토랑 리뷰 조회 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return None
    except Exception as e:
        print(f"❌ 레스토랑 리뷰 조회 중 오류: {str(e)}")
        return None

# 레스토랑 리뷰 조회 실행
restaurant_reviews = test_get_restaurant_reviews()


✅ 레스토랑 리뷰 조회 성공
   레스토랑 ID: res_1234
   총 리뷰 수: 8개

   리뷰 1:
     ID: rev_3006
     내용: 서비스는 기대 안 하는 게 정신 건강에 좋다. 하지만 음식은 나쁘지 않아요....
     사용자: user_2006

   리뷰 2:
     ID: rev_3004
     내용: 음식은 맛있을 때도 있지만 오늘은 좀 짰다. 조금 아쉬웠어요....
     사용자: user_2004

   리뷰 3:
     ID: rev_3012
     내용: 배치 리뷰 3012입니다. 맛있어요!...
     사용자: user_3012


## 12. 배치 감성 분석 테스트


In [14]:
# 배치 감성 분석 테스트
def test_sentiment_analysis_batch():
    """배치 감성 분석 테스트"""
    try:
        response = requests.post(
            f"{BASE_URL}/api/v1/sentiment/analyze/batch",
            json=[
                {
                    "reviews": [
                        "맛있어요!",
                        "좋아요!",
                        "별로예요"
                    ],
                    "restaurant_name": "비즐",
                    "restaurant_id": "res_1234"
                },
                {
                    "reviews": [
                        "괜찮아요",
                        "만족스러워요",
                        "아쉬워요"
                    ],
                    "restaurant_name": "시올돈",
                    "restaurant_id": "res_1235"
                }
            ]
        )
        
        if response.status_code == 200:
            results = response.json()
            print("✅ 배치 감성 분석 성공")
            print(f"   처리된 레스토랑 수: {len(results)}개")
            
            for result in results:
                print(f"\n   {result['restaurant_name']} ({result['restaurant_id']}):")
                print(f"     긍정 비율: {result['positive_ratio']}%")
                print(f"     부정 비율: {result['negative_ratio']}%")
                print(f"     총 리뷰: {result['total_count']}개")
            
            return results
        else:
            print(f"❌ 배치 감성 분석 실패: {response.status_code}")
            print(f"   오류: {response.text}")
            return None
    except Exception as e:
        print(f"❌ 배치 감성 분석 중 오류: {str(e)}")
        return None

# 배치 감성 분석 실행
batch_sentiment_result = test_sentiment_analysis_batch()


✅ 배치 감성 분석 성공
   처리된 레스토랑 수: 2개

   비즐 (res_1234):
     긍정 비율: 67%
     부정 비율: 33%
     총 리뷰: 3개

   시올돈 (res_1235):
     긍정 비율: 67%
     부정 비율: 33%
     총 리뷰: 3개


## 13. 전체 테스트 요약

모든 테스트를 한 번에 실행하고 결과를 요약합니다.


In [15]:
# 전체 테스트 요약
def run_all_tests():
    """모든 테스트를 실행하고 결과를 요약"""
    print("=" * 60)
    print("전체 API 테스트 시작")
    print("=" * 60)
    
    test_results = {}
    
    # 1. 헬스 체크
    print("\n[1] 헬스 체크")
    test_results['health'] = check_health()
    
    # 2. 벡터 데이터 업로드
    print("\n[2] 벡터 데이터 업로드")
    test_results['upload'] = upload_vector_data(sample_data)
    
    # 3. 감성 분석
    print("\n[3] 감성 분석")
    test_results['sentiment'] = test_sentiment_analysis() is not None
    
    # 4. 벡터 검색
    print("\n[4] 벡터 검색")
    test_results['search'] = test_vector_search() is not None
    
    # 5. 리뷰 요약
    print("\n[5] 리뷰 요약")
    test_results['summarize'] = test_summarize() is not None
    
    # 6. 강점 추출
    print("\n[6] 강점 추출")
    test_results['strengths'] = test_extract_strengths() is not None
    
    # 7. 리뷰 Upsert
    print("\n[7] 리뷰 Upsert")
    test_results['upsert'] = test_upsert_review() is not None
    
    # 8. 배치 Upsert
    print("\n[8] 배치 Upsert")
    test_results['batch_upsert'] = test_upsert_reviews_batch() is not None
    
    # 9. 리뷰 삭제
    print("\n[9] 리뷰 삭제")
    test_results['delete'] = test_delete_review() is not None
    
    # 10. 배치 삭제
    print("\n[10] 배치 삭제")
    test_results['batch_delete'] = test_delete_reviews_batch() is not None
    
    # 11. 이미지 리뷰 검색
    print("\n[11] 이미지 리뷰 검색")
    test_results['images'] = test_search_reviews_with_images() is not None
    
    # 12. 레스토랑 리뷰 조회
    print("\n[12] 레스토랑 리뷰 조회")
    test_results['restaurant_reviews'] = test_get_restaurant_reviews() is not None
    
    # 13. 배치 감성 분석
    print("\n[13] 배치 감성 분석")
    test_results['batch_sentiment'] = test_sentiment_analysis_batch() is not None
    
    # 결과 요약
    print("\n" + "=" * 60)
    print("테스트 결과 요약")
    print("=" * 60)
    
    total_tests = len(test_results)
    passed_tests = sum(1 for v in test_results.values() if v)
    failed_tests = total_tests - passed_tests
    
    for test_name, result in test_results.items():
        status = "✅ 통과" if result else "❌ 실패"
        print(f"   {test_name:20s}: {status}")
    
    print(f"\n   총 테스트: {total_tests}개")
    print(f"   통과: {passed_tests}개")
    print(f"   실패: {failed_tests}개")
    print(f"   성공률: {(passed_tests/total_tests*100):.1f}%")
    
    return test_results

# 전체 테스트 실행 (주석 해제하여 실행)
test_summary = run_all_tests()


전체 API 테스트 시작

[1] 헬스 체크
✅ API 서버가 정상적으로 실행 중입니다.
   상태: {'status': 'healthy', 'version': '1.0.0'}

[2] 벡터 데이터 업로드
✅ 벡터 데이터 업로드 성공
   포인트 수: 9
   컬렉션: reviews_collection

[3] 감성 분석
✅ 감성 분석 성공
   긍정 비율: 50%
   부정 비율: 50%
   긍정 개수: 3
   부정 개수: 3
   총 개수: 6
   LLM 재분류: 4개

[4] 벡터 검색
✅ 벡터 검색 성공
   검색 결과: 5개

   결과 1:
     리뷰: 웨이팅이 길 줄 알았는데 회전이 빨라서 괜찮았다. 맛도 좋고 가성비도 좋아요!...
     점수: 0.654
     레스토랑: 비즐

   결과 2:
     리뷰: 점심시간이라 사람이 많았지만 생각보다 빨리 나왔다. 맛도 좋고 서비스도 친절해요!...
     점수: 0.590
     레스토랑: 비즐

   결과 3:
     리뷰: 음식은 맛있을 때도 있지만 오늘은 좀 짰다. 조금 아쉬웠어요....
     점수: 0.572
     레스토랑: 비즐

[5] 리뷰 요약


KeyboardInterrupt: 